In [1]:
import pandas as pd
import pickle
from tqdm.notebook import tqdm
import nltk
from nltk import word_tokenize, sent_tokenize, pos_tag
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from collections import Counter
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/hjkim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/hjkim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/hjkim/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
# pickle 파일 불러오기

# Abstract
with open('../entity_extraction/Preprocessed_Data/H01L_2020-2022_9585_abstract_keyphrases.pickle', 'rb') as fr:
    abstract_keyphrases = pickle.load(fr)
with open('../entity_extraction/Preprocessed_Data/H01L_2020-2022_9585_abstract_keyphrases_list.pickle', 'rb') as fr:
    abstract_keyphrases_list = pickle.load(fr)
    
# Verbs
with open('result/verb_abstract_lemmatized_reduced.pickle', 'rb') as fr:
    verbs = pickle.load(fr)

In [36]:
verbs_list = [tup[0] for tup in verbs]
keyphrases_list = [tup[0] for tup in abstract_keyphrases_list]

lemmatizer = WordNetLemmatizer()
pos_list = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'] # 동사

In [46]:
# 한 문장 안에서 인접하고 서로 다른 keyphrase와 keyphrase 사이에 동사가 존재하는 경우 triplet으로 추출

triplets = []

for abstract in tqdm(abstract_keyphrases):
    for i in range(len(abstract['keyphrases']) - 1): # 인접한 keyphrase들을 iterate
        a = abstract['keyphrases'][i]
        b = abstract['keyphrases'][i + 1]
        if a['keyphrase'] != b['keyphrase']:
#             print(a['keyphrase'], '--',b['keyphrase'])
#             print(abstract['abstract'][a['end_index']:b['start_index']])
            text_within = abstract['abstract'][a['end_index']:b['start_index']] # 두 keyphrase 사이의 텍스트
            words_within = pos_tag(word_tokenize(text_within))
            if '.' not in [i[0] for i in words_within]: # 같은 문장인 경우만 고려
                for word in words_within:
                    if word[1] in pos_list: # 동사만 추출
                        triplets.append([a['keyphrase'], lemmatizer.lemmatize(word[0], 'v'), b['keyphrase']])

  0%|          | 0/9585 [00:00<?, ?it/s]

In [51]:
len(triplets)

68247

In [48]:
triplets

[['wafer polishing system', 'comprise', 'slurry distribution unit'],
 ['slurry distribution unit', 'mount', 'slurry tank'],
 ['slurry distribution unit', 'polish', 'slurry tank'],
 ['slurry distribution unit', 'distribute', 'slurry tank'],
 ['slurry distribution unit', 'flow', 'slurry tank'],
 ['slurry distribution unit', 'polish', 'slurry tank'],
 ['slurry tank', 'connect', 'slurry pump'],
 ['slurry tank', 'store', 'slurry pump'],
 ['slurry pump', 'connect', 'first circulation line'],
 ['slurry pump', 'polish', 'first circulation line'],
 ['slurry pump', 'transfer', 'first circulation line'],
 ['slurry pump', 'polish', 'first circulation line'],
 ['first circulation line', 'be', 'second circulation line'],
 ['first circulation line', 'connect', 'second circulation line'],
 ['second circulation line', 'be', 'sl'],
 ['second circulation line', 'connect', 'sl'],
 ['second circulation line', 'be', 'sl'],
 ['second circulation line', 'connect', 'sl'],
 ['polishing method', 'use', 'polishin

In [53]:
# Create a dictionary to store the counts
list_counts = defaultdict(int)

# Count the occurrences of each list
for sublist in triplets:
    list_counts[tuple(sublist)] += 1

# Print the counts
for sublist, count in list_counts.items():
    print(f"List {sublist} appears {count} times.")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [59]:
from collections import defaultdict

# Create a dictionary to store the counts
list_counts = defaultdict(int)

# Count the occurrences of each list
for sublist in triplets:
    list_counts[tuple(sublist)] += 1

# Sort the dictionary items by count in descending order
sorted_counts = sorted(list_counts.items(), key=lambda x: x[1], reverse=True)

# Print the top results
for sublist, count in sorted_counts[:100]:
#     print(f"List {sublist} appears {count} times.")
    print(f"{count} times : {sublist}")

63 times : ('semiconductor device', 'include', 'semiconductor substrate')
63 times : ('semiconductor device', 'include', 'substrate')
34 times : ('semiconductor device', 'include', 'gate structure')
21 times : ('substrate processing apparatus', 'include', 'process chamber')
21 times : ('semiconductor device', 'include', 'fin structure')
17 times : ('semiconductor device', 'include', 'dielectric layer')
17 times : ('semiconductor structure', 'include', 'substrate')
14 times : ('semiconductor device', 'include', 'first')
14 times : ('semiconductor device', 'form', 'semiconductor substrate')
13 times : ('chemical mechanical polishing pad', 'polish', 'tungsten')
12 times : ('semiconductor structure', 'include', 'semiconductor substrate')
12 times : ('bias power applying unit', 'apply', 'phase adjuster')
11 times : ('substrate processing apparatus', 'include', 'substrate holding unit')
11 times : ('semiconductor device', 'form', 'dielectric layer')
11 times : ('electrostatic chuck', 'includ

## Save

In [63]:
with open('result/triplet_count.pickle','wb') as fw:
    pickle.dump(sorted_counts, fw)

## Load

In [2]:
import pickle

with open('result/triplet_count.pickle', 'rb') as fr:
    triplet_count = pickle.load(fr)

In [4]:
len(triplet_count)

56862

In [8]:
triplet_count[:10]

[(('semiconductor device', 'include', 'semiconductor substrate'), 63),
 (('semiconductor device', 'include', 'substrate'), 63),
 (('semiconductor device', 'include', 'gate structure'), 34),
 (('substrate processing apparatus', 'include', 'process chamber'), 21),
 (('semiconductor device', 'include', 'fin structure'), 21),
 (('semiconductor device', 'include', 'dielectric layer'), 17),
 (('semiconductor structure', 'include', 'substrate'), 17),
 (('semiconductor device', 'include', 'first'), 14),
 (('semiconductor device', 'form', 'semiconductor substrate'), 14),
 (('chemical mechanical polishing pad', 'polish', 'tungsten'), 13)]

## Dataframe으로

In [16]:
list_A = [tup[0][0] for tup in triplet_count]
list_Verb = [tup[0][1] for tup in triplet_count]
list_B = [tup[0][2] for tup in triplet_count]
Count = [tup[1] for tup in triplet_count]

In [17]:
print(len(list_A))
print(len(list_Verb))
print(len(list_B))
print(len(Count))

56862
56862
56862
56862


In [18]:
import pandas as pd

df = pd.DataFrame(columns=['A', 'Verb', 'B', 'Count'])
df['A'] = list_A
df['Verb'] = list_Verb
df['B'] = list_B
df['Count'] = Count

df

,A,Verb,B,Count
0,semiconductor device,include,semiconductor substrate,63
1,semiconductor device,include,substrate,63
2,semiconductor device,include,gate structure,34
3,substrate processing apparatus,include,process chamber,21
4,semiconductor device,include,fin structure,21
...,...,...,...,...
56857,firing elements,configure,metallization layer,1
56858,firing elements,fire,metallization layer,1
56859,user interface,comprise,control circuitry,1
56860,control circuitry,configure,firing elements,1


In [51]:
df_verb = df.groupby('Verb', as_index=False).sum().sort_values('Count', ascending=False).reset_index(drop=True)
df_verb

,Verb,A,B,Count
0,be,hydroxy groupareadoped glass layermixing manif...,hydrogen atommaintenance areaisolation portion...,10261
1,include,semiconductor devicesemiconductor devicesemico...,semiconductor substratesubstrategate structure...,7301
2,form,semiconductor devicesemiconductor devicesemico...,semiconductor substratedielectric layergate st...,6239
3,have,first columnar portioncentral openingcylindric...,first film thicknesscover memberexhaust gaspar...,3314
4,configure,seal capcarrier waferprocessing chamberprocess...,insulating unitcolor emitting LEDsmass flow co...,1935
...,...,...,...,...
1283,lifting-off,bonding,pickup substrate,1
1284,lifting/lowering,rotating cam 31,engagement latch connection portion 357,1
1285,liquefy,exhaust system,iron oxide,1
1286,liquid-treat,liquid treatment chamber,drying chamber,1


## Save

In [54]:
df.to_csv('result/triplet_df.csv', index=False)
df_verb.to_csv('result/triplet_verb_df.csv', index=False)

## Load

In [55]:
import pandas as pd

df_loaded = pd.read_csv('result/triplet_df.csv')
df_verb_loaded = pd.read_csv('result/triplet_verb_df.csv')

In [56]:
df_loaded

,A,Verb,B,Count
0,semiconductor device,include,semiconductor substrate,63
1,semiconductor device,include,substrate,63
2,semiconductor device,include,gate structure,34
3,substrate processing apparatus,include,process chamber,21
4,semiconductor device,include,fin structure,21
...,...,...,...,...
56857,firing elements,configure,metallization layer,1
56858,firing elements,fire,metallization layer,1
56859,user interface,comprise,control circuitry,1
56860,control circuitry,configure,firing elements,1


In [57]:
df_verb_loaded

,Verb,A,B,Count
0,be,hydroxy groupareadoped glass layermixing manif...,hydrogen atommaintenance areaisolation portion...,10261
1,include,semiconductor devicesemiconductor devicesemico...,semiconductor substratesubstrategate structure...,7301
2,form,semiconductor devicesemiconductor devicesemico...,semiconductor substratedielectric layergate st...,6239
3,have,first columnar portioncentral openingcylindric...,first film thicknesscover memberexhaust gaspar...,3314
4,configure,seal capcarrier waferprocessing chamberprocess...,insulating unitcolor emitting LEDsmass flow co...,1935
...,...,...,...,...
1283,lifting-off,bonding,pickup substrate,1
1284,lifting/lowering,rotating cam 31,engagement latch connection portion 357,1
1285,liquefy,exhaust system,iron oxide,1
1286,liquid-treat,liquid treatment chamber,drying chamber,1
